<a href="https://colab.research.google.com/github/Kingaahad/Analyze-and-Extract/blob/main/function_calling_projec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-2.0-flash-exp',verbose=True)

llm.invoke("Hi, I am Daniyal Asghar")

AIMessage(content="Hi Daniyal Asghar, it's nice to meet you! How can I help you today? Are you working on something, looking for information, or just saying hello? Let me know!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8f04b755-6006-4b85-a1f6-65048f96d921-0', usage_metadata={'input_tokens': 9, 'output_tokens': 41, 'total_tokens': 50, 'input_token_details': {'cache_read': 0}})

In [4]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return (a * b) + 50

llm_with_tools = llm.bind_tools([multiply])

In [5]:
tools_list=[multiply]
model_with_tools = llm.bind_tools(tools_list)

In [6]:
result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing great, thanks for asking! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4a8b1a72-95a0-486c-8e0a-947bf53cafb5-0', usage_metadata={'input_tokens': 68, 'output_tokens': 18, 'total_tokens': 86, 'input_token_details': {'cache_read': 0}})

In [ ]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result


In [7]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)


In [8]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)


<ipython-input-8-f11f73b77070>:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [11]:
# Query the agent
response1 = agent.run("What is 105 * 20?")  # Uses the tool
print(response1)





> Entering new AgentExecutor chain...
I need to multiply 105 by 20. I don't have a multiplication tool, but I can use the addition tool to add 105 twenty times, which is the same as multiplication. However, that is inefficient. Instead, I will multiply 105 by 2 first then multiply the result by 10. Multiplying by 2 is the same as adding the number to itself.
Action: AdditionTool
Action Input: 105 105
Observation: The result is: 210
Thought:I have 105 * 2 = 210. Now I need to multiply 210 by 10. Multiplying by 10 is the same as adding a zero to the end of the number.
Final Answer: 2100

> Finished chain.
2100
